# Module 4: PySpark Joins - Marketing Campaign Analysis
**Scenario:** Working for an E-Commerce Giant (e.g., Amazon, Flipkart, Target).

**Objective:** Combine Customer Data + Transaction Data to finding "High Value Customers" and "Inactive Customers".

**The Data Engineering Context:**
In real systems, data is *Normalized* (split into tables to save space).
*   **Table A:** Customer Profile (ID, Name, Email, Country) - 50 Million rows.
*   **Table B:** Orders (OrderID, CustomerID, Amount, Date) - 2 Billion rows.
*   **Your Job:** JOIN them efficiently to answer: *"Who spent more than $500 last month?"*

---
## 1. Setup Environment

In [ ]:
# Setup PySpark
try:
    import pyspark
    print("PySpark is already installed")
except ImportError:
    print("Installing PySpark...")
    !pip install pyspark findspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Marketing_Campaign_Joins") \
    .master("local[*]") \
    .getOrCreate()

print("Spark Session Ready")

## 2. Load Two Datasets (Customers & Transactions)
We simulate the two tables commonly found in SQL databases.

*   **Customer Table:** Key = `customer_id`
*   **Transaction Table:** Key = `customer_id` (Foreign Key)

In [ ]:
# --- 1. Customer Profiles ---
# Notice: CUST_005 (Grace) exists but has NO transactions.
customers = [
    ("CUST_001", "Alice", "alice@email.com", "USA"),
    ("CUST_002", "Bob", "bob@email.com", "UK"),
    ("CUST_003", "Charlie", "charlie@email.com", "USA"),
    ("CUST_004", "David", "david@email.com", "India"),
    ("CUST_005", "Grace", "grace@email.com", "Canada")
]
df_customers = spark.createDataFrame(customers, ["customer_id", "name", "email", "country"])

# --- 2. Transactions ---
# Notice: TXN regarding 'CUST_999' exists but that customer is NOT in our profile table (Data Error?).
transactions = [
    ("T1", "CUST_001", 120.0, "2023-01-01"),
    ("T2", "CUST_001", 50.0, "2023-01-05"),
    ("T3", "CUST_002", 500.0, "2023-01-10"),
    ("T4", "CUST_003", 40.0, "2023-01-12"),
    ("T5", "CUST_999", 100.0, "2023-01-15") # Orphan Record
]
df_transactions = spark.createDataFrame(transactions, ["txn_id", "customer_id", "amount", "txn_date"])

print("--- Customers ---")
df_customers.show()

print("--- Transactions ---")
df_transactions.show()

## 3. Inner Join: "Show me details of customers who BOUGHT something"
*   **Inner Join:** Returns rows where there is a match in BOTH tables.
*   **Result:** Alice, Bob, Charlie.
*   **Excluded:** Grace (No purchase), CUST_999 (Identity unknown).

**Why use it?** When building a "Sales Dashboard" where missing names are useless.

In [ ]:
# The Join Syntax: df1.join(df2, on="key", how="type")
df_inner = df_customers.join(df_transactions, "customer_id", "inner")

print("--- INNER JOIN (Only Matching Records) ---")
df_inner.show()
# Notice: No Grace, No CUST_999. Only valid customers who bought something.

## 4. Left Join: "Find Churned / Inactive Customers"
*   **Left Join:** Keep ALL customers (Left Table), and bring transaction info if it exists.
*   **If no match?** Fills with `NULL`.
*   **Business Use:** "Send an email coupon to anyone who registered but didn't buy anything."

**Goal:** Find customers where `txn_id` IS NULL.

In [ ]:
# 1. Perform Left Join
df_left = df_customers.join(df_transactions, "customer_id", "left")

print("--- LEFT JOIN (All Customers, Matches or Nulls) ---")
df_left.show()
# Look at Grace (CUST_005). Her txn_id is null.

# 2. Filter for Inactive Customers
df_inactive = df_left.filter(col("txn_id").isNull()) \
    .select("customer_id", "name", "email") # Get email to send campaign

print("--- Campaign Target List: Inactive Customers ---")
df_inactive.show()
# This list goes to the Mailing Team!

## 5. Right Join / Full Outer Join: Investigating Data Anomalies
*   **Right Join:** Keep everything in Transactions (Right Table).
*   **Why?** To find "Orphan Records" - transactions made by customers who don't exist in our Customer Database. This points to a **Data Integrity Issue** (Bug?).

**Goal:** Find transactions where `name` is NULL.

In [ ]:
# 1. Perform Right Join (Transactions is the Right table)
df_right = df_customers.join(df_transactions, "customer_id", "right")

print("--- RIGHT JOIN (All Transactions, Matches or Nulls) ---")
df_right.show()
# Notice: CUST_999 has amount $100.0, but 'name' is null.

# 2. Filter for Data Anomalies (Orphan transactions)
df_orphans = df_right.filter(col("name").isNull())

print("--- Data Quality Issue: Transactions without Valid Customer ---")
df_orphans.show()
# This list goes to the Database Admin team to fix!

## 6. Advanced: Identifying "High Value Customers" (VIPs)
1.  **Join** Customers & Transactions (Inner).
2.  **Group By** Customer.
3.  **Sum** the Amount.
4.  **Filter** Total Spend > $400.
5.  **Sort** Descending.

In [ ]:
# 1. Join & Aggregation
df_vip_analysis = df_customers.join(df_transactions, "customer_id", "inner") \
    .groupBy("customer_id", "name", "country") \
    .agg(
        sum("amount").alias("total_spend"),
        count("txn_id").alias("txn_count")
    )

print("--- Total Spend Per Customer ---")
df_vip_analysis.show()

# 2. Filter for VIPs (> 400 total spend)
df_vips = df_vip_analysis.filter(col("total_spend") > 400).orderBy(col("total_spend").desc())

print("--- VIP Customers (Send 'Thank You' Gift) ---")
df_vips.show()
# Expect Bob to be #1 with $500.